In [29]:
import torch
import torch.nn as nn
# from numpy import exp, sqrt
from numpy.random import normal
from torch import exp, sqrt, randn_like
from rdkit import Chem
import torch.nn.functional as F

In [2]:
from torch_geometric.datasets import Planetoid
import os.path as osp
import inspect
import torch_geometric.transforms as T
src_file_path = inspect.getfile(lambda: None)
path = osp.join(osp.dirname(osp.realpath(src_file_path)), '..', 'data', 'Planetoid')
dataset = Planetoid(path, 'Cora', transform=T.NormalizeFeatures())

In [85]:
from torch_geometric.utils import train_test_split_edges

cora = dataset[0]
cora.train_mask = cora.val_mask = cora.test_mask = None
cora = train_test_split_edges(cora)
cora.train_pos_edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

In [6]:
type(dataset[0])

torch_geometric.data.data.Data

In [25]:
# runs out of memory, will need CPU for full set
data = train_r_data.data
data.train_mask = data.val_mask = data.test_mask = None
data = train_test_split_edges(data=data, val_ratio=0.05, test_ratio=0.05)
data.train_pos_edge_index


In [24]:
from ts_vae.data_processors.grambow_processor import ReactionDataset

base_path = r'data/'

train_r_data = ReactionDataset(base_path, geo_file = 'train_r') 
train_ts_data = ReactionDataset(base_path, geo_file = 'train_ts')
train_p_data = ReactionDataset(base_path, geo_file = 'train_p')  

test_r_data = ReactionDataset(base_path, geo_file = 'test_r') 
test_ts_data = ReactionDataset(base_path, geo_file = 'test_ts')
test_p_data = ReactionDataset(base_path, geo_file = 'test_p')

In [23]:
# simple GCN VAE

# right now, using the edge features, but will need node positions (data.pos) later on
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.nn import VGAE

# simple VariationalGCNEncoder
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True)
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_log_var = GCNConv(2 * out_channels, out_channels, cached=True)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_log_var(x, edge_index)

num_features = train_r_data.num_features
out_channels = 4

model = VGAE(VariationalGCNEncoder(num_features, out_channels))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# data = train_r_data.data
# x = data.x.to(device)
# train_pos_edge_index = data.train_pos_edge_index.to(device) # not working rn
# optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [39]:
from torch_geometric.data import DataLoader
train_r_loader = DataLoader(train_r_data, batch_size=1, shuffle=False)

In [37]:
for data in loader:
    x = data.to(device)


KeyboardInterrupt: 

In [40]:
train_r_data.data

Data(edge_attr=[173336, 4], edge_index=[2, 173336], idx=[6739], pos=[89300, 3], x=[89300, 11], z=[89300])

In [ ]:
# how do x, edge_index (connectivity), edge_attr, pos change together?

In [43]:
train_r_data.data.num_nodes

89300

In [ ]:
def train(model, loader, optimiser, loss, device, task): # TODO: schedulersc
    
    model.train()
    
    # this iterates through each molecule of dataset (as its PTG graph features)
    for data in tqdm(loader, total=len(loader)):
        
        # x = node feature matrix
        x = data.x.to(device)
        # data = data.to(device)
        optimiser.zero_grad()

        z = model.encode(x)

        loss = model.recon_loss(z, train_pos_edge_index)
        loss = loss + (1 / data.num_nodes) * model.kl_loss()
        loss.backward()

        optimiser.step()


        out = model(data)
        result = loss(out, standardiser())


In [82]:
def train():
    model.train()
    optimiser.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimiser.step()
    return float(loss)

def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

epochs = 3
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

NameError: name 'train_pos_edge_index' is not defined

In [75]:
data

Data(edge_attr=[173336, 4], edge_index=[2, 173336], idx=[6739], pos=[89300, 3], x=[89300, 11], z=[89300])

In [71]:
train_r_data.data.x

tensor([[0., 1., 0.,  ..., 0., 0., 3.],
        [0., 1., 0.,  ..., 0., 0., 2.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

In [27]:
# lucky way

from torch_geometric.nn import MessagePassing

class GCNConv(MessagePassing):
    def __init__(self, args):
        super(GCNConv, self).__init__(aggr='add')
        self.linear = nn.Linear(args.)

In [ ]:
# way given by link under class

# basic GCNConv

from torch.nn import Dropout, ReLU
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, InMemoryDataset

class StandardGCN(torch.nn.Module):
    """ Standard implementation of GCN with GCNConv layers. Code mostly lifted from https://github.com/klicperajo/gdc/blob/master/models.py
        Used to evaluate 2D molecular graph created by our .sdf data as a baseline against proposed 3D methods.
    """

    def __init__(self, geometry_dataset: InMemoryDataset, hidden: List[int] = 2, dropout: float = 0.5):
        super(GCN, self).__init__()

        num_features = [geometry_dataset.num_featuresdata.x.shape[1]] + hidden + [geometry_dataset.num_classes]
        layers = []
        for in_features, out_features in zip(num_features[:-1], num_features[1:]):
            layers.append(GCNConv(in_features, out_features))
        self.layers = ModuleList(layers)
        
        self.dropout = Dropout(p=dropout)
        self.activation_func = ReLU()

    def reset_parameters(self):
        for layer in self.layers:
            layer.reset_parameters()
    
    def forward(self, data: Data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        for i, layer in enumerate(self.layers):
            x = layer(x, edge_index, edge_weight=edge_attr)
            if i == len(self.layers) - 1:
                break
            x = self.activation_func(x)
            x = self.dropout(x)
        return torch.nn.functional.log_softmax(x, dim=1)

In [25]:
test_r_data.data.x.shape

torch.Size([11202, 11])

In [26]:
test_r_data.num_features

11

In [ ]:
# loss for R, P latent spaces
    # frame func as generate R, P latent spaces
# loss for TS decoding 

# related: DimeNet, GNNExplainer, SchNet 
# VAEs: VGAE, ARGVA

In [ ]:
# https://github.com/rusty1s/pytorch_geometric/blob/master/examples/autoencoder.py
def train():
    # train
    # use optimiser
    # encode
    # decode to TS
    # evaluate loss compared to expected TS
    # loss.backward() 
    # optimiser.step()
    # return float(loss)

# run test_r and test_p through encoder and decode to TS, eval against test_ts
def test():
    return

In [2]:
LATENT_DIM = 4      # arbitrary
MAX_SIZE = 22*3     # try first with (x,y,z) for each atom

class MoleculeVAE(nn.Module):
    def __init__(self): # maybe also init encoder, decoder, in_channels, out_channels, depth, hidden size, dropout, gnn_type
        # self.decoder = DefaultDecoder() if decoder is None else decoder
        # maybe pass in data as reaction frame here too?
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(MAX_SIZE, 32),
            nn.ReLU(),
            nn.Linear(32, 2*LATENT_DIM)
        )
        self.decoder = nn.Sequential(
            nn.Linear(2*LATENT_DIM, 32),
            nn.ReLU(),
            nn.Linear(32, MAX_SIZE)
        )

        """
        self.convs = torch.nn.ModuleList()

        for _ in range(self.depth):
            if self.gnn_type == ?:
                self.convs.append(e.g. GCNConv(args))
        """
    
    def reparameterise(self, mean_z, log_var_z):
        if self.training:
            eps = randn_like() # is this meant to be randn_like(log_var_z)?
            z = mean_z + eps * sqrt(exp(log_var_z))
            return z
        else:
            return mean_z
    
    def forward(self, x): # note: might be easier to do this the other way explicitly defining mean and var
        # reshape input into a vector, then reshape using view(-1, batchsize=2, d)
        params_z = self.encoder(x.view(-1, input_size)).view(-1, 2, LATENT_DIM) # this encoder may need to be changed
        mean_z = params_z[:, 0, :]
        log_var_z = params_z[:, 1, :]
        z = self.reparameterise(mean_z, log_var_z)
        return self.decoder(z), mean_z, log_var_z 

    # x_hat = model TS; x = TS
    def loss_func(generated_TS, real_TS, mean_z, log_var_z, beta=1):
        BCE = nn.functional.binary_cross_entropy(generated_TS, real_TS.view(-1, MAX_SIZE), reduction='sum') 
        KLD = 0.5 * torch.sum(exp(log_var_z) - log_var_z - 1 + mean_z**2)
        return BCE + beta * KLD

# define device and model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MoleculeVAE().to(device)

In [1]:
from ts_vae.data_processors.grambow_processor import ReactionDataset

base_path = r'data/'

train_r_data = ReactionDataset(base_path, geo_file = 'train_r') 
train_ts_data = ReactionDataset(base_path, geo_file = 'train_ts')
train_p_data = ReactionDataset(base_path, geo_file = 'train_p')  

test_r_data = ReactionDataset(base_path, geo_file = 'test_r') 
test_ts_data = ReactionDataset(base_path, geo_file = 'test_ts')
test_p_data = ReactionDataset(base_path, geo_file = 'test_p')

In [4]:
# either take in reaction dataset or take in .sdf?
# might be better to keep these functions inside the original processor
class EDA():
    
    def __init__(self):
        pass

    # TODO
    def visualise_feature_dynamics(self):
        # takes in reactants, ts, products; calculates reaction centre
        # compare how reaction centre changes: by how much, what precision do we need?
        # could do similar with interatomic distances
        return

# model run class
class ModelRun():
    def __init__(self, training_rxns, test_rxns, model):
        self.training_rxns = training_rxns
        self.test_rxns = test_rxns

    # plot loss 
    # plotting evaluation here
    
    def preprocess_data():
        # preprocess the data for each model as it suits
        return
    
    # compare model TS: average, initial guesses, final estimates, reals
        # will allow me to compare several models against each other
    
    # will have funcs to evaluate structure of output
        # for generative models: evaluate structure of latent space
        # evaluate how R and P are combined

    # how are TS formed?



In [13]:
def create_dataloader(args):

    if isinstance(modes, str):
        modes = [modes]


def construct_loader(args, modes=('train', 'val')):

    if isinstance(modes, str):
        modes = [modes]

    data_df = pd.read_csv(args.data_path)

    smiles = data_df.iloc[:, 0].values
    labels = data_df.iloc[:, 1].values.astype(np.float32)

    loaders = []
    for mode in modes:
        dataset = MolDataset(smiles, labels, args, mode)
        loader = DataLoader(dataset=dataset,
                            batch_size=args.batch_size,
                            shuffle=not args.no_shuffle if mode == 'train' else False,
                            num_workers=args.num_workers,
                            pin_memory=True,
                            sampler=StereoSampler(dataset) if args.shuffle_pairs else None)
        loaders.append(loader)

    if len(loaders) == 1:
        return loaders[0]
    else:
        return loaders

list

In [13]:
# setting optimiser
learning_rate = 1e-3
optimiser = torch.optim.Adam(model.parameters(), lr = learning_rate)

# training and testing the VAE
epochs = 5
codes = dict(mean=list(), log_var=list(), y=list())
for epoch in range(0, epochs+1):
    # training
    if epoch > 0:
        model.train()
        train_loss = 0
        for real_TS, _ in train_loader:
            real_TS = real_TS.to(device)
            # === forward ===
            generated_TS, mean_z, log_var_z = model(x)
            loss = loss_func(generated_TS, real_TS, mean_z, log_var_z)
            train_loss += loss.item()
            # === backward ===
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
        # === log ===
        print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')
        
    # testing
    means, log_vars, labels = list(), list(), list()
    with torch.no_grad():
        model.eval()
        test_loss = 0
        for x, y in test_loader:
            x = x.to(device)
            # === forward ===
            x_hat, mean, log_var = model(x)
            test_loss += loss_function(x_hat, x, mean, log_var).item()
            # === log ===
            means.append(mean.detach())
            log_vars.append(log_var.detach())
            labels.append(y.detach())
    # === log ===
    codes['mean'].append(torch.cat(means))
    codes['log_var'].append(torch.cat(log_vars))
    codes['y'].append(torch.cat(labels))
    test_loss /= len(test_loader.dataset)
    print(f'===> Test set loss: {test_loss:.4f}')
    display_images(x, x_hat, 1, f'Epoch {epoch}')

NameError: name 'test_loader' is not defined

In [ ]:
class VAE(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Sequential(
            # linear (size of input, 2d), size of input= max possible size i.e. largest mol
            nn.Linear(input_size, d**2),
            nn.ReLU(),
            nn.Linear(d ** 2, d * 2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(d, d ** 2),
            nn.ReLU(),
            nn.Linear(d ** 2, input_size)
            # would use sigmoid here if input was between 0 and 1
        )

    def reparameterise(self, mean_z, log_var_z):
        if self.training:
            # eps = normal(loc=0, scale=1, size=(len(graphs.nodes), self.latent_dim=2d))
            # since variances only positive, computing log allows you to output full real range for encoder
            eps = normal(0, 1, size=(len(input_nodes), latent_dims=2d))
            z = mean_z + eps * sqrt(exp(log_var_z))
            return z
        else:
            return mean_z

    def forward(self, x):
        # reshape input into a vector, then reshape using view(-1, batchsize=2, d)
        params_z = self.encoder(x.view(-1, input_size)).view(-1, 2, d)
        
        mean_z = params_z[:, 0, :]
        log_var_z = params_z[:, 1, :]
        z = self.reparameterise(mean_z, log_var_z)
        return self.decoder(z), mean_z, log_var_z 

model = VAE().to(device)

In [ ]:
# setting optimiser
learning_rate = 1e-3
optimiser = torch.optim.Adam(model.parameters(), lr = learning_rate)

# reconstruction + KL divergence losses summed over all elements
def loss_function(x_hat, x, mean_z, log_var_z, beta):
    # binary cross entropy between input and reconstruction
    BCE = nn.functional.binary_cross_entropy(x_hat, x.view(-1, 784), reduction='sum') 
    # kl divergence: var is linear, - log var is logarithmic, mean is squared 
    KLD = 0.5 * torch.sum(exp(log_var_z) - log_var_z - 1 + mean_z**2)
    return BCE + beta * KLD

In [ ]:
# training and testing the VAE
epochs = 5
codes = dict(mean=list(), log_var=list(), y=list())
for epoch in range(0, epochs+1):
    # training
    if epoch > 0:
        model.train()
        train_loss = 0
        for x, _ in train_loader:
            x = x.to(device)
            # === forward ===
            x_hat, mean, log_var = model(x)
            loss = loss_function(x_hat, x, mean, log_var)
            train_loss += loss.item()
            # === backward ===
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
        # === log ===
        print(f'====> Epoch: {epoch} Average loss: {
            train_loss / len(train_loader.dataset):.4f}')
        
    # testing
    means, log_vars, labels = list(), list(), list()
    with torch.no_grad():
        model.eval()
        test_loss = 0
        for x, y in test_loader:
            x = x.to(device)
            # === forward ===
            x_hat, mean, log_var = model(x)
            test_loss += loss_function(x_hat, x, mean, log_var).item()
            # === log ===
            means.append(mean.detach())
            log_vars.append(log_var.detach())
            labels.append(y.detach())
    # === log ===
    codes['mean'].append(torch.cat(means))
    codes['log_var'].append(torch.cat(log_vars))
    codes['y'].append(torch.cat(labels))
    test_loss /= len(test_loader.dataset)
    print(f'===> Test set loss: {test_loss:.4f}')
    display_images(x, x_hat, 1, f'Epoch {epoch}')


In [ ]:
# generating a few samples
N = 16
z = torch.randn((N, d)).to(device)
sample = model.decoder(z)
display_images(None, sample, N//4, count=True)

# Choose starting and ending point for the interpolation -> shows original and reconstructed

A, B = 1, 14
sample = model.decoder(torch.stack((mean[A].data, mean[B].data), 0))
display_images(None, torch.stack(((
    x[A].data.view(-1),
    x[B].data.view(-1),
    sample.data[0],
    sample.data[1]
)), 0))